In [51]:
import math
from math import log
import numpy as np
import sys
from typing import List, Tuple
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
import strings
from importlib import reload

reload(arrays)
reload(trees)
reload(lists)
reload(graphs)
reload(search)
import arrays as ar
import trees as tr
import lists as ll
import graphs as g
import strings as s
from lists import Node

print()

In [111]:
def count_bits(a: List[int]) -> int:
    n, l = len(a), int(log(max(a), 2)) + 1
    bits = lambda b: sum(map(lambda x: not x & (1 << b), a))
    return sum((n - f) * f for f in map(bits, range(l + 1))) * 2 % 1000000007

In [112]:
count_bits([1, 3, 5])

8

In [90]:
~1

-2